In [1]:
import pandas as pd
import os
from math import sqrt
import glob

In [2]:
def distancia_euclidiana(lat1, lon1, lat2, lon2):
    return sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2) * 111

In [3]:
pasta_geocoded = r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos Processados"
pasta_finais = r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos Finais"
arquivo_estacoes = r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos Auxiliares\Locais de Coleta.xlsx"

os.makedirs(pasta_finais, exist_ok=True)

est_df = pd.read_excel(arquivo_estacoes)

In [4]:
RAIOS_KM = [1.5, 5, 10, 20]

arquivos_geocoded = glob.glob(os.path.join(pasta_geocoded, "*_GEOCODED.*"))

In [5]:
if not arquivos_geocoded:
    print("Nenhum arquivo _GEOCODED encontrado!")
else:
    for caminho in arquivos_geocoded:
        arquivo = os.path.basename(caminho)
        # Lê dependendo da extensão
        if arquivo.lower().endswith(".xlsx"):
            df_base = pd.read_excel(caminho)
        elif arquivo.lower().endswith(".csv"):
            df_base = pd.read_csv(caminho, encoding='latin1', sep=';')
        else:
            print(f"Ignorado (extensão não suportada): {arquivo}")
            continue

        df_base = df_base.rename(columns=lambda x: x.strip().upper())

        if not ({"LATITUDE", "LONGITUDE"}.issubset(df_base.columns) and ("LOCAL" in df_base.columns or "ENDEREÇO" in df_base.columns)):
            print(f"Arquivo ignorado (colunas ausentes): {arquivo}")
            continue

        # PARA CADA RAIO, GERAR UM ARQUIVO
        for RAIO_KM in RAIOS_KM:

            df = df_base.copy()

            df['Estação atribuída'] = None
            df['Distância (km)'] = None

            for idx, local in df.iterrows():
                lat_l = local['LATITUDE']
                lon_l = local['LONGITUDE']

                if pd.isna(lat_l) or pd.isna(lon_l):
                    continue

                distancias = [(est['Estação'], distancia_euclidiana(lat_l, lon_l, est['LAT'], est['LONG']))
                            for _, est in est_df.iterrows()]

                dentro_raio = [(nome, d) for nome, d in distancias if d <= RAIO_KM]

                if len(dentro_raio) == 0:
                    estacao_final = "sem estação correspondente"
                    dist_final = None
                elif len(dentro_raio) == 1:
                    estacao_final = dentro_raio[0][0]
                    dist_final = dentro_raio[0][1]
                else:
                    estacao_final, dist_final = min(dentro_raio, key=lambda x: x[1])

                df.at[idx, 'Estação atribuída'] = estacao_final
                df.at[idx, 'Distância (km)'] = None if dist_final is None else round(dist_final, 4)

            # Nomeia o arquivo de acordo com o raio
            nome_final = os.path.splitext(arquivo)[0] + f"_FINAL_{RAIO_KM}km.xlsx"
            caminho_saida = os.path.join(pasta_finais, nome_final)
            df.to_excel(caminho_saida, index=False)

        print(f"Arquivos gerados para {arquivo} com os 4 raios:", RAIOS_KM)


Arquivo ignorado (colunas ausentes): BOMBEIROS_GEOCODED.csv
Arquivos gerados para BOMBEIROS_GEOCODED.xlsx com os 4 raios: [1.5, 5, 10, 20]
Arquivos gerados para Locais_por_Data_GEOCODED.xlsx com os 4 raios: [1.5, 5, 10, 20]
Arquivos gerados para RELATOS_GEOCODED.xlsx com os 4 raios: [1.5, 5, 10, 20]
